In [1]:
#
# LSTM모델을 이용해서 주가예측
#
import os
import settings
import pandas as pd        # 라이브러리
import numpy as np         # 라이브러리
from keras.models import Sequential      # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import Dense           # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import LSTM            # 딥러닝을 구동하는 데 필요한 케라스 함수
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
from sklearn.metrics import mean_squared_error
import pymysql         # 파이썬에서 mysql연동시켜주는 라이브러리

#
# DB테이블 값 조회 (SELECT)
#
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')

# 오차율 가져오기
try:
    with connection.cursor() as cursor:
        sql = "select accuracy from stock_hye WHERE company_name='LG'"
        cursor.execute("set names utf8")
        cursor.execute(sql)
        result_accuracy = cursor.fetchone()
        
        for i in result_accuracy:
            accuracy = i
            
finally:
    connection.close()


# 데이터셋 생성 함수
look_back = 1
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
 
#
# 저장되어있는 주식데이터 불러오기
#
sydtpath = os.path.join(settings.BASE_DIR, 'chart_data/%s' % (settings.get_today_str()))
stock_code = "lg"
fullpath = sydtpath + os.path.sep + stock_code + '.csv'
pandf = pd.read_csv(fullpath, index_col="Date")

# 데이터 전처리
nparr = pandf['Close'].values[1:]     # 맨처음 'Close'데이터부터 차례대로 nparr에 저장
print(nparr)
nparr.astype('float32')    # float형으로 변환
print(nparr)
nparr = nparr.reshape(-1,1)
print(nparr)
 
# 정규화 (0~1사이의 값으로 바꿔준다)
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
 
# 학습용, 테스트용 데이터로 나누기 (90%를 학습용 데이터, 10%를 테스트용 데이터)
train_size = int(len(nptf) * 0.9)
test_size = len(nptf) - train_size
train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
print(len(train), len(test))
 
# 학습을 위한 데이터셋 생성하기 (학습용, 테스트용으로 구분)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
 
# RNN모델은 3차원 데이터
# trainX, testX값을 [samples, time steps, features] 형태로 reshape
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
 
# LSTM모델
model = Sequential()                                                # 딥러닝 구조, 층을 설정
model.add(LSTM(20, input_shape=(1, 1)))     # (timestep, feature)   # 층이 추가됨 (add)   # 입력층, 첫번째 은닉층                   
model.add(Dense(1))                                                 # 출력층 (하나)
# 모델을 컴파일 (컴퓨터가 알아들을 수 있도록)   # 오차함수, 최적화 방법
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])   # metrics : 모델 수행 결과를 나타내게 설정 (과적합 문제 방지)
model.fit(trainX, trainY, epochs=1000, batch_size=400, verbose=2)      # 모델을 실제로 수행    # batch_size : 전체 데이터를 400개씩 사용
                                                                                               # verbose(로깅)  2 : epoch당 나오게

# 예측값 평가하기 (얼마나 정확한지)
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)                 # testPredict : 예측 값
testY = scaler.inverse_transform(testY)                             # testY : 실제 값
testScore = math.sqrt(mean_squared_error(testY, testPredict))       # mean_squared_error : 평균 제곱근 오차
print('Train Score: %.2f RMSE' % testScore)                         # 예측 값과 실제 값 차이 출력
 
# 예측 데이터 출력
lastX = nptf[-1]
lastX = np.reshape(lastX, (1, 1, 1))
lastY = model.predict(lastX)
predict = scaler.inverse_transform(lastY)                    # 정규화 시킨 값을 역변환
print('Predict the Close value of final day: %d' % predict)  # 데이터 입력 마지막 다음날 종가 예측
    
# 차트출력, 저장
font_path = "C:/Windows/Fonts/a옛날목욕탕L.ttf"
fontprop = fm.FontProperties(fname=font_path, size=15)

plt.plot(testPredict)
plt.plot(testY)

plt.title('[LG] 예측 그래프\n예측값 : %d \t\t\t\t\t\t\t\t오차율 : %s' % (predict, accuracy), fontproperties=fontprop)

#plt.savefig("./chart_picture/lg.png",dpi=300)
plt.savefig("C:\source\SPF\chart_picture\lg.png", dpi=300)
#plt.show()

# 사진 데이터 binary형식으로 바꿔주는 함수
def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData


# 
# DB테이블 값 삽입 (INSERT)
#
# MySQL Connection 연결
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')
try:
    with connection.cursor() as cursor:
        sql = 'INSERT INTO stock_hye (company_name, stock_price, image) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE stock_price = VALUES(stock_price), image = VALUES(image)'
        image = convertToBinaryData("C:\source\SPF\chart_picture\lg.png")
        cursor.execute(sql, ('LG', int(predict), image))          # 넣으려는 값
    connection.commit()
    
finally:
    connection.close()

Using TensorFlow backend.


[45950 47600 48700 ... 71500 72200 71100]
[45950 47600 48700 ... 71500 72200 71100]
[[45950]
 [47600]
 [48700]
 ...
 [71500]
 [72200]
 [71100]]
2314 258


C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/1000
 - 1s - loss: 0.2201 - acc: 4.3253e-04
Epoch 2/1000
 - 0s - loss: 0.2034 - acc: 4.3253e-04
Epoch 3/1000
 - 0s - loss: 0.1872 - acc: 4.3253e-04
Epoch 4/1000
 - 0s - loss: 0.1715 - acc: 4.3253e-04
Epoch 5/1000
 - 0s - loss: 0.1563 - acc: 4.3253e-04
Epoch 6/1000
 - 0s - loss: 0.1418 - acc: 4.3253e-04
Epoch 7/1000
 - 0s - loss: 0.1278 - acc: 4.3253e-04
Epoch 8/1000
 - 0s - loss: 0.1144 - acc: 4.3253e-04
Epoch 9/1000
 - 0s - loss: 0.1016 - acc: 4.3253e-04
Epoch 10/1000
 - 0s - loss: 0.0895 - acc: 4.3253e-04
Epoch 11/1000
 - 0s - loss: 0.0782 - acc: 4.3253e-04
Epoch 12/1000
 - 0s - loss: 0.0677 - acc: 4.3253e-04
Epoch 13/1000
 - 0s - loss: 0.0581 - acc: 4.3253e-04
Epoch 14/1000
 - 0s - loss: 0.0494 - acc: 4.3253e-04
Epoch 15/1000
 - 0s - loss: 0.0417 - acc: 4.3253e-04
Epoch 16/1000
 - 0s - loss: 0.0350 - acc: 4.3253e-04
Epoch 17/1000
 - 0s - loss: 0.0294 - acc: 4.3253e-04
Epoch 18/1000
 - 0s - loss: 0.0247 - acc: 8.6505e-04
Epoch 19/1000
 - 0s - loss: 0.0209 - acc: 8.6505e-04
Ep

Epoch 154/1000
 - 0s - loss: 7.3756e-04 - acc: 8.6505e-04
Epoch 155/1000
 - 0s - loss: 7.2319e-04 - acc: 8.6505e-04
Epoch 156/1000
 - 0s - loss: 7.0928e-04 - acc: 8.6505e-04
Epoch 157/1000
 - 0s - loss: 6.9666e-04 - acc: 8.6505e-04
Epoch 158/1000
 - 0s - loss: 6.8442e-04 - acc: 8.6505e-04
Epoch 159/1000
 - 0s - loss: 6.7265e-04 - acc: 8.6505e-04
Epoch 160/1000
 - 0s - loss: 6.6176e-04 - acc: 8.6505e-04
Epoch 161/1000
 - 0s - loss: 6.5138e-04 - acc: 8.6505e-04
Epoch 162/1000
 - 0s - loss: 6.4187e-04 - acc: 8.6505e-04
Epoch 163/1000
 - 0s - loss: 6.3278e-04 - acc: 8.6505e-04
Epoch 164/1000
 - 0s - loss: 6.2410e-04 - acc: 8.6505e-04
Epoch 165/1000
 - 0s - loss: 6.1593e-04 - acc: 8.6505e-04
Epoch 166/1000
 - 0s - loss: 6.0851e-04 - acc: 8.6505e-04
Epoch 167/1000
 - 0s - loss: 6.0138e-04 - acc: 8.6505e-04
Epoch 168/1000
 - 0s - loss: 5.9487e-04 - acc: 8.6505e-04
Epoch 169/1000
 - 0s - loss: 5.8863e-04 - acc: 8.6505e-04
Epoch 170/1000
 - 0s - loss: 5.8258e-04 - acc: 8.6505e-04
Epoch 171/1000

 - 0s - loss: 5.0307e-04 - acc: 8.6505e-04
Epoch 296/1000
 - 0s - loss: 5.0300e-04 - acc: 8.6505e-04
Epoch 297/1000
 - 0s - loss: 5.0297e-04 - acc: 8.6505e-04
Epoch 298/1000
 - 0s - loss: 5.0304e-04 - acc: 8.6505e-04
Epoch 299/1000
 - 0s - loss: 5.0301e-04 - acc: 8.6505e-04
Epoch 300/1000
 - 0s - loss: 5.0279e-04 - acc: 8.6505e-04
Epoch 301/1000
 - 0s - loss: 5.0263e-04 - acc: 8.6505e-04
Epoch 302/1000
 - 0s - loss: 5.0262e-04 - acc: 8.6505e-04
Epoch 303/1000
 - 0s - loss: 5.0251e-04 - acc: 8.6505e-04
Epoch 304/1000
 - 0s - loss: 5.0245e-04 - acc: 8.6505e-04
Epoch 305/1000
 - 0s - loss: 5.0255e-04 - acc: 8.6505e-04
Epoch 306/1000
 - 0s - loss: 5.0223e-04 - acc: 8.6505e-04
Epoch 307/1000
 - 0s - loss: 5.0221e-04 - acc: 8.6505e-04
Epoch 308/1000
 - 0s - loss: 5.0218e-04 - acc: 8.6505e-04
Epoch 309/1000
 - 0s - loss: 5.0206e-04 - acc: 8.6505e-04
Epoch 310/1000
 - 0s - loss: 5.0245e-04 - acc: 8.6505e-04
Epoch 311/1000
 - 0s - loss: 5.0214e-04 - acc: 8.6505e-04
Epoch 312/1000
 - 0s - loss: 

Epoch 437/1000
 - 0s - loss: 4.9171e-04 - acc: 8.6505e-04
Epoch 438/1000
 - 0s - loss: 4.9147e-04 - acc: 8.6505e-04
Epoch 439/1000
 - 0s - loss: 4.9159e-04 - acc: 8.6505e-04
Epoch 440/1000
 - 0s - loss: 4.9130e-04 - acc: 8.6505e-04
Epoch 441/1000
 - 0s - loss: 4.9116e-04 - acc: 8.6505e-04
Epoch 442/1000
 - 0s - loss: 4.9155e-04 - acc: 8.6505e-04
Epoch 443/1000
 - 0s - loss: 4.9102e-04 - acc: 8.6505e-04
Epoch 444/1000
 - 0s - loss: 4.9095e-04 - acc: 8.6505e-04
Epoch 445/1000
 - 0s - loss: 4.9084e-04 - acc: 8.6505e-04
Epoch 446/1000
 - 0s - loss: 4.9082e-04 - acc: 8.6505e-04
Epoch 447/1000
 - 0s - loss: 4.9077e-04 - acc: 8.6505e-04
Epoch 448/1000
 - 0s - loss: 4.9074e-04 - acc: 8.6505e-04
Epoch 449/1000
 - 0s - loss: 4.9065e-04 - acc: 8.6505e-04
Epoch 450/1000
 - 0s - loss: 4.9063e-04 - acc: 8.6505e-04
Epoch 451/1000
 - 0s - loss: 4.9041e-04 - acc: 8.6505e-04
Epoch 452/1000
 - 0s - loss: 4.9028e-04 - acc: 8.6505e-04
Epoch 453/1000
 - 0s - loss: 4.9004e-04 - acc: 8.6505e-04
Epoch 454/1000

 - 0s - loss: 4.7845e-04 - acc: 8.6505e-04
Epoch 579/1000
 - 0s - loss: 4.7825e-04 - acc: 8.6505e-04
Epoch 580/1000
 - 0s - loss: 4.7851e-04 - acc: 8.6505e-04
Epoch 581/1000
 - 0s - loss: 4.7823e-04 - acc: 8.6505e-04
Epoch 582/1000
 - 0s - loss: 4.7831e-04 - acc: 8.6505e-04
Epoch 583/1000
 - 0s - loss: 4.7797e-04 - acc: 8.6505e-04
Epoch 584/1000
 - 0s - loss: 4.7797e-04 - acc: 8.6505e-04
Epoch 585/1000
 - 0s - loss: 4.7776e-04 - acc: 8.6505e-04
Epoch 586/1000
 - 0s - loss: 4.7809e-04 - acc: 8.6505e-04
Epoch 587/1000
 - 0s - loss: 4.7768e-04 - acc: 8.6505e-04
Epoch 588/1000
 - 0s - loss: 4.7766e-04 - acc: 8.6505e-04
Epoch 589/1000
 - 0s - loss: 4.7738e-04 - acc: 8.6505e-04
Epoch 590/1000
 - 0s - loss: 4.7737e-04 - acc: 8.6505e-04
Epoch 591/1000
 - 0s - loss: 4.7768e-04 - acc: 8.6505e-04
Epoch 592/1000
 - 0s - loss: 4.7707e-04 - acc: 8.6505e-04
Epoch 593/1000
 - 0s - loss: 4.7710e-04 - acc: 8.6505e-04
Epoch 594/1000
 - 0s - loss: 4.7702e-04 - acc: 8.6505e-04
Epoch 595/1000
 - 0s - loss: 

Epoch 720/1000
 - 0s - loss: 4.6701e-04 - acc: 8.6505e-04
Epoch 721/1000
 - 0s - loss: 4.6688e-04 - acc: 8.6505e-04
Epoch 722/1000
 - 0s - loss: 4.6702e-04 - acc: 8.6505e-04
Epoch 723/1000
 - 0s - loss: 4.6779e-04 - acc: 8.6505e-04
Epoch 724/1000
 - 0s - loss: 4.6720e-04 - acc: 8.6505e-04
Epoch 725/1000
 - 0s - loss: 4.6675e-04 - acc: 8.6505e-04
Epoch 726/1000
 - 0s - loss: 4.6658e-04 - acc: 8.6505e-04
Epoch 727/1000
 - 0s - loss: 4.6682e-04 - acc: 8.6505e-04
Epoch 728/1000
 - 0s - loss: 4.6797e-04 - acc: 8.6505e-04
Epoch 729/1000
 - 0s - loss: 4.6727e-04 - acc: 8.6505e-04
Epoch 730/1000
 - 0s - loss: 4.6725e-04 - acc: 8.6505e-04
Epoch 731/1000
 - 0s - loss: 4.6613e-04 - acc: 8.6505e-04
Epoch 732/1000
 - 0s - loss: 4.6639e-04 - acc: 8.6505e-04
Epoch 733/1000
 - 0s - loss: 4.6572e-04 - acc: 8.6505e-04
Epoch 734/1000
 - 0s - loss: 4.6639e-04 - acc: 8.6505e-04
Epoch 735/1000
 - 0s - loss: 4.6601e-04 - acc: 8.6505e-04
Epoch 736/1000
 - 0s - loss: 4.6638e-04 - acc: 8.6505e-04
Epoch 737/1000

 - 0s - loss: 4.6075e-04 - acc: 8.6505e-04
Epoch 862/1000
 - 0s - loss: 4.6172e-04 - acc: 8.6505e-04
Epoch 863/1000
 - 0s - loss: 4.6421e-04 - acc: 8.6505e-04
Epoch 864/1000
 - 0s - loss: 4.6298e-04 - acc: 8.6505e-04
Epoch 865/1000
 - 0s - loss: 4.6125e-04 - acc: 8.6505e-04
Epoch 866/1000
 - 0s - loss: 4.6199e-04 - acc: 8.6505e-04
Epoch 867/1000
 - 0s - loss: 4.6177e-04 - acc: 8.6505e-04
Epoch 868/1000
 - 0s - loss: 4.6034e-04 - acc: 8.6505e-04
Epoch 869/1000
 - 0s - loss: 4.5956e-04 - acc: 8.6505e-04
Epoch 870/1000
 - 0s - loss: 4.6037e-04 - acc: 8.6505e-04
Epoch 871/1000
 - 0s - loss: 4.6016e-04 - acc: 8.6505e-04
Epoch 872/1000
 - 0s - loss: 4.6106e-04 - acc: 8.6505e-04
Epoch 873/1000
 - 0s - loss: 4.6167e-04 - acc: 8.6505e-04
Epoch 874/1000
 - 0s - loss: 4.5998e-04 - acc: 8.6505e-04
Epoch 875/1000
 - 0s - loss: 4.6007e-04 - acc: 8.6505e-04
Epoch 876/1000
 - 0s - loss: 4.6022e-04 - acc: 8.6505e-04
Epoch 877/1000
 - 0s - loss: 4.6050e-04 - acc: 8.6505e-04
Epoch 878/1000
 - 0s - loss: 